# Monte Carlo Project

## Iacob Jessica, Mourre Grégoire

In thus Jupter file, we will implement the different methods presented and described in https://github.com/gregoiremrr/Monte-Carlo-for-American-Options.


In [150]:
import numpy as np
from numpy.random import default_rng

# Longstaff and Schwartz method for the lower bound

In [151]:
def phi(x):
    return np.array([1, x, x**2], dtype=object)
l = len(phi(0))

def reg(x, alpha):
    return np.sum(alpha*phi(x))

n = 10**4
m = 20
rng = default_rng()

r, sigma, x, K, T = 0, .2, 100, 100, 1
dt = T/m

def g(x, K=K):
    return (x-K) * (x >= K)

In [152]:
# Tsitsiklis & Van Roy's algorithm

# Step 1
B = rng.normal(size=m*n).reshape(m,n)
X = np.cumprod(np.concatenate([[x*np.ones(n)], 1 + r * dt + sigma * np.sqrt(dt) * B]), axis=0)

# Step 2
V = np.zeros(n*(m+1)).reshape(m+1,n)
V[-1,:] = g(X[-1,:])

# Step 3
from scipy.optimize import minimize
alpha0 = np.zeros(l)
for i in range(m-1, 0, -1):
    alpha_star = minimize(lambda alpha: np.mean((V[i+1,:] - reg(X[i,:], alpha))**2), alpha0).x
    _1 = g(X[i,:])
    _2 = reg(X[i,:], alpha_star)
    V[i,:] = _1 * (_1 >= _2) + _2 * (_1 < _2)

# Step 4
_1 = g(x)
_2 = np.mean(V[1,:])
V0 = _1 * (_1 > _2) + _2 * (_1 <= _2)

print("Estimator:", V0)

Estimator: 9.420865858094977


In [153]:
# Longstaff & Schwartz's algorithm

# Step 2
V2 = np.zeros(n*(m+1)).reshape((m+1),n)
V2[-1,:] = g(X[-1,:])

alpha_tau = np.zeros(l*(m+1)).reshape((m+1),l)

# Step 3
alpha0 = np.zeros(l)
for i in range(m-1, 0, -1):
    alpha_tau[i,:] = minimize(lambda alpha: np.mean((V2[i+1,:] - reg(X[i,:], alpha))**2), alpha0).x
    _1 = g(X[i,:])
    _2 = reg(X[i,:], alpha_tau[i,:])
    V2[i,:] = _1 * (_1 >= _2) + V2[(i+1),:] * (_1 < _2)

# Step 4 (not used)
_1 = g(x)
_2 = np.mean(V2[1,:])
V02 = np.mean(_1 * (_1 > _2) + V2[1,:] * (_1 <= _2))

# Step 5
B2 = rng.normal(size=m*n).reshape(m,n)
X2 = np.cumprod(np.concatenate([[x*np.ones(n)], 1 + r * dt + sigma * np.sqrt(dt) * B2]), axis=0)

V3 = np.zeros(n*(m+1)).reshape((m+1),n)
V3[-1,:] = g(X2[-1,:])

for i in range(m-1, 0, -1):
    _1 = g(X2[i,:])
    _2 = reg(X2[i,:], alpha_tau[i,:])
    V3[i,:] = _1 * (_1 >= _2) + V3[(i+1),:] * (_1 < _2)

_1 = g(x)
_2 = np.mean(V3[1,:])
V03_ = _1 * (_1 > _2) + V3[1,:] * (_1 <= _2)

V03 = np.mean(V03_)
std = np.std(V03_)
conv_interval = V03 + np.array([-1,1]) * 1.96 * std / np.sqrt(n)

print("Estimator:", V03)
print("Standard deviation:", std / np.sqrt(n))
print("Condidence interval 95%:", conv_interval)
print("Error:", 100 * 1.96 * std / (V03 * np.sqrt(n)), "%")

Estimator: 7.703291271565558
Standard deviation: 0.10204914340688298
Condidence interval 95%: [7.50327495 7.90330759]
Error: 2.5965047150143765 %


# Martingales from Approximate Value Functions

In [154]:
n2 = n

B_upperbound = rng.normal(size=m)
X_upperbound = np.cumprod(np.concatenate([[x], 1 + r * dt + sigma * np.sqrt(dt) * B_upperbound]), axis=0)
Normal = rng.normal(size=m*n2).reshape(m,n2)
M = np.zeros(m+1)
Mi = 0

for i in range(1, m):
    V_upperbound = max(g(X_upperbound[i]), reg(X_upperbound[i], alpha_tau[i,:]))
    X_next = X_upperbound[i-1] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[i-1,:])
    V_Ynext = np.maximum(g(X_next), reg(X_next, alpha_tau[i,:]))
    delta = V_upperbound - np.mean(V_Ynext)
    Mi += delta
    M[i] = Mi

X_next = X_upperbound[-2] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[-1,:])
delta = g(X_upperbound[-1]) - np.mean(g(X_next))
Mi += delta
M[-1] = Mi

V0_upperbound = np.max(g(X_upperbound) - M)

print(V0_upperbound)

8.533557622086267


# Martingales from Stopping Rules

In [155]:
n2 = n

Normal = rng.normal(size=m*n*n2).reshape(m,n,n2)
M = np.zeros(n*(m+1)).reshape(n,m+1)
Mi = 0 #??

for i in range(1, m):
    V_upperbound = max(g(X_upperbound[i]), reg(X_upperbound[i], alpha_tau[i,:]))
    X_next = X_upperbound[i-1] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[i-1,:])
    V_Ynext = np.maximum(g(X_next), reg(X_next, alpha_tau[i,:]))
    delta = V_upperbound - np.mean(V_Ynext)
    Mi += delta
    M[i] = Mi

X_next = X_upperbound[-2] * (1 + r * dt + sigma * np.sqrt(dt) * Normal[-1,:])
delta = g(X_upperbound[-1]) - np.mean(g(X_next))
Mi += delta
M[-1] = Mi

V0_upperbound = np.max(g(X_upperbound) - M)

print(V0_upperbound)

7.964040011602146


# Finite difference